In [ ]:
#!git clone https://github.com/hyperconnect/TC-ResNet.git
#!pip3 install -r ./TC-ResNet/requirements/py36-gpu.txt

In [ ]:
#cd TC-ResNet/

In [ ]:
# train model
#! ./scripts/commands/TCResNet8Model-1.0_mfcc_40_3010_0.001_mom_l1.sh

In [1]:
!git clone https://github.com/tranHieuDev23/TC-ResNet.git

Cloning into 'TC-ResNet'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 21 (delta 6), reused 20 (delta 5), pack-reused 0
Unpacking objects: 100% (21/21), done.


In [2]:
cd TC-ResNet/

/content/TC-ResNet


Uncomment following 5 cells to load google speech dataset

In [3]:
#!mkdir dataset

In [4]:
#cd dataset/

In [5]:
#!wget http://download.tensorflow.org/data/speech_commands_v0.01.tar.gz

In [6]:
#!tar xvf speech_commands_v0.01.tar.gz

In [7]:
#cd ..

In [8]:
from os import listdir
from os.path import join, isfile, isdir, normpath
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
import numpy as np
from itertools import zip_longest
import librosa
import random
import math
import numpy as np
from random import seed, randint
from train import get_tc_resnet_8, get_tc_resnet_14
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.models import Model, Sequential
from keras.layers import Input, Conv1D, ReLU, BatchNormalization, Add, AveragePooling1D, Dense, Flatten, Dropout, Activation
from google.colab import files


In [9]:
# To train entire tc-resnet--uncomment
"""
from process_data import process_file, generate_noisy_sample
def __load_audio_filenames_with_class__(root_folder):
    classes = [item for item in listdir(root_folder) if isdir(
        join(root_folder, item)) and not item.startswith('_')]
    filenames = []
    class_ids = []
    for i in range(len(classes)):
        c = classes[i]
        class_filenames = __load_audio_filenames__(join(root_folder, c))
        filenames.extend(class_filenames)
        class_ids.extend([i] * len(class_filenames))
    return filenames, class_ids, classes


def __load_audio_filenames__(root_folder):
    filenames = []
    for entry in listdir(root_folder):
        full_path = join(root_folder, entry)
        if (isfile(full_path)):
            if (entry.endswith('.wav')):
                filenames.append(full_path)
        else:
            filenames.extend(__load_audio_filenames__(full_path))
        if (len(filenames) >= 100):
            break
    return filenames


def __load_subset_filenames__(root_folder, filename):
    subset_list = []
    with open(join(root_folder, filename)) as f:
        for line in f:
            line = line.strip()
            if (len(line) == 0):
                continue
            subset_list.append(normpath(join(root_folder, line)))
    return set(subset_list)


def load_data_from_folder(root_folder):
    filenames, class_ids, classes = __load_audio_filenames_with_class__(root_folder)
    dataset_size = len(filenames)
    X_train = []
    y_train = []
    X_test = []
    y_test = []
    X_validation = []
    y_validation = []
    pool = Pool(cpu_count() - 1)
    for (results, filepath, class_id, random_roll) in tqdm(pool.imap_unordered(process_file, zip_longest(filenames, class_ids)), total=dataset_size):
        filepath = normpath(filepath)
        is_testing = 1 <= random_roll and random_roll <= 10
        is_validation = 11 <= random_roll and random_roll <= 20
        for item in results:
            if (is_testing):
                X_test.append(item)
                y_test.append(class_id)
            elif (is_validation):
                X_validation.append(item)
                y_validation.append(class_id)
            else:
                X_train.append(item)
                y_train.append(class_id)
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    X_validation = np.array(X_validation)
    y_validation = np.array(y_validation)
    return X_train, y_train, X_test, y_test, X_validation, y_validation, classes

X_train, y_train, X_test, y_test, X_validation, y_validation, classes = load_data_from_folder('dataset')
num_classes = len(classes)
print(num_classes)
(num_train, input_length, num_channel) = X_train.shape
num_test = X_test.shape[0]
num_validation = X_validation.shape[0]
model_14 = get_tc_resnet_8((input_length, num_channel), num_classes, 1.5)
model_14.compile(optimizer=Adam(),loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#checkpoint_cb = ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', save_weights_only=True, period=5)
model_14.fit(x=X_train, y=y_train, batch_size=1024, epochs=100, validation_data=(X_test, y_test))#, callbacks=[checkpoint_cb])
print(model_14.evaluate(X_validation, y_validation))
model_14.save_weights('weights.h5')
files.download('weights.h5') 
"""

"\nfrom process_data import process_file, generate_noisy_sample\ndef __load_audio_filenames_with_class__(root_folder):\n    classes = [item for item in listdir(root_folder) if isdir(\n        join(root_folder, item)) and not item.startswith('_')]\n    filenames = []\n    class_ids = []\n    for i in range(len(classes)):\n        c = classes[i]\n        class_filenames = __load_audio_filenames__(join(root_folder, c))\n        filenames.extend(class_filenames)\n        class_ids.extend([i] * len(class_filenames))\n    return filenames, class_ids, classes\n\n\ndef __load_audio_filenames__(root_folder):\n    filenames = []\n    for entry in listdir(root_folder):\n        full_path = join(root_folder, entry)\n        if (isfile(full_path)):\n            if (entry.endswith('.wav')):\n                filenames.append(full_path)\n        else:\n            filenames.extend(__load_audio_filenames__(full_path))\n        if (len(filenames) >= 100):\n            break\n    return filenames\n\n\nde

In [11]:
# Make a bottle neck model

original_model    = get_tc_resnet_8((321, 40), 30, 1.5) #model corresponding to kws on google speech cmds: input length, num_channel, num_classes
original_model.load_weights('weights.h5') #Assuming this file is loaded in the current working dir
bottleneck_input  = original_model.get_layer(index=0).input
print(bottleneck_input)
bottleneck_output = original_model.get_layer(index=-2).output
print(bottleneck_output)
bottleneck_model  = Model(inputs=bottleneck_input,outputs=bottleneck_output)

KerasTensor(type_spec=TensorSpec(shape=(None, 321, 40), dtype=tf.float32, name='input_2'), name='input_2', description="created by layer 'input_2'")
KerasTensor(type_spec=TensorSpec(shape=(None, 2808), dtype=tf.float32, name=None), name='dropout_1/Identity:0', description="created by layer 'dropout_1'")


In [12]:
# Add the last softmax layer
for layer in bottleneck_model.layers:
    layer.trainable = False

new_model = Sequential()
new_model.add(bottleneck_model)
kws_classes = 101
new_model.add(Dense(kws_classes, activation="softmax", input_dim=2808))

Following 5 cells to load the Competition dataset (train)

In [13]:
!mkdir dataset1

In [14]:
cd dataset1/

/content/TC-ResNet/dataset1


In [15]:
!gdown --id 1w4Bn038sLxuv9PKswk2KorCUmgPNup3c

Downloading...
From: https://drive.google.com/uc?id=1w4Bn038sLxuv9PKswk2KorCUmgPNup3c
To: /content/TC-ResNet/dataset1/train.zip
211MB [00:01, 189MB/s]


In [ ]:
!unzip train.zip

In [17]:
cd ..

/content/TC-ResNet


Load the _background_noise folder in dataset1/train


In [18]:

AUDIO_LENGTH = 2

seed(163)


def __load_background_noises__(root_folder):
    noises = []
    noise_folder = join(root_folder, '_background_noise_')
    for item in listdir(noise_folder):
        if (not item.endswith('.wav')):
            continue
        samples, sr = librosa.load(join(noise_folder, item), sr=None)
        noises.append(samples)
    return noises


noises = __load_background_noises__('')


def generate_noisy_sample(samples, noise):
    samples_length = len(samples)
    noise_length = len(noise)
    if (noise_length < samples_length):
        return samples
    noise_start = random.randint(0, noise_length - samples_length - 1)
    noise_part = noise[noise_start:noise_start + samples_length]
    noise_coeff = random.uniform(0.0, 0.1)
    audio_offset = math.floor(
        random.uniform(-samples_length * 0.1, samples_length * 0.1))
    new_samples = np.zeros((samples_length))
    if (audio_offset >= 0):
        new_samples[audio_offset:] = samples[:samples_length - audio_offset]
    else:
        new_samples[:samples_length + audio_offset] = samples[-audio_offset:]
    new_samples = noise_part * noise_coeff + \
        (1.0 - noise_coeff) * new_samples
    return new_samples


def get_mfcc(samples, sr):
    return librosa.feature.mfcc(samples, sr=sr, n_mfcc=40, n_fft=400, hop_length=100).transpose()


def process_file(argv):
    (filepath, class_id) = argv
    results = []
    samples, sr = librosa.load(filepath, sr=None)
    samples_len = len(samples)
    if (samples_len > sr * AUDIO_LENGTH):
        samples = samples[- sr * AUDIO_LENGTH:]
    elif (samples_len < sr * AUDIO_LENGTH):
        temp = np.zeros((sr * AUDIO_LENGTH))
        temp[:samples_len] = samples
        samples = temp
    mfcc = get_mfcc(samples, sr)
    results.append(mfcc)
    random_roll = randint(1, 100)
    is_testing = 1 <= random_roll and random_roll <= 10
    is_validation = 11 <= random_roll and random_roll <= 20
    if (not is_testing and not is_validation):
        for item in noises:
            new_samples = generate_noisy_sample(samples, item)
            mfcc = get_mfcc(new_samples, sr)
            results.append(mfcc)
    return results, filepath, class_id, random_roll


In [19]:
def __load_new_audio_filenames_with_class__(root_folder):
    classes = [item for item in listdir(root_folder) if item.startswith('T')] #classes as "T00xx"
    classes.sort()
    classes.append('others')
    filenames = []
    class_ids = []
    for i in range(len(classes)-1):
        c = classes[i] 
        class_filenames = __load_new_audio_filenames__((join(root_folder, c, "enrollment"))) #location of wav files for kws
        filenames.extend(class_filenames)
        class_ids.extend([i] * len(class_filenames))
        class_filenames = __load_new_audio_filenames__((join(root_folder, c, "others"))) #location of wav files for kws
        filenames.extend(class_filenames)
        class_ids.extend([len(classes)-1] * len(class_filenames))        
    return filenames, class_ids, classes


def __load_new_audio_filenames__(root_folder):
    filenames = []
    for entry in listdir(root_folder):
        full_path = join(root_folder, entry)
        if (isfile(full_path)):
            if (entry.endswith('.wav')):
                filenames.append(full_path)
        else:
            filenames.extend(__load_audio_filenames__(full_path))
        #if (len(filenames) >= 10):
            #break
    return filenames

def load_new_data_from_folder(root_folder):
    filenames, class_ids, classes = __load_new_audio_filenames_with_class__(root_folder)
    dataset_size = len(filenames)
    X_train = []
    y_train = []
    X_test = []
    y_test = []
    X_validation = []
    y_validation = []
    pool = Pool(cpu_count() - 1)
    for (results, filepath, class_id, random_roll) in tqdm(pool.imap_unordered(process_file, zip_longest(filenames, class_ids)), total=dataset_size):
        filepath = normpath(filepath)
        is_testing = 1 <= random_roll and random_roll <= 10 #vary these to modify train-val-test lengths
        is_validation = 11 <= random_roll and random_roll <=20
        for item in results:
            if (is_testing):
                X_test.append(item)
                y_test.append(class_id)
            elif (is_validation):
                X_validation.append(item)
                y_validation.append(class_id)
                X_test.append(item)
                y_test.append(class_id)
            else:
                X_train.append(item)
                y_train.append(class_id)
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    X_validation = np.array(X_validation)
    y_validation = np.array(y_validation)
    return X_train, y_train, X_test, y_test, X_validation, y_validation, classes

X_train, y_train, X_test, y_test, X_validation, y_validation, classes = load_new_data_from_folder('dataset1/train')
num_classes = len(classes)
(num_train, input_length, num_channel) = X_train.shape
num_test = X_test.shape[0]
num_validation = X_validation.shape[0]
print(num_classes)
print(num_train)
print(num_test)
print(num_validation)


100%|██████████| 3000/3000 [02:52<00:00, 17.42it/s]


101
17129
553
288


In [20]:
new_model.compile(optimizer=Adam(),loss='sparse_categorical_crossentropy', metrics=['accuracy'])
new_model.fit(x=X_train, y=y_train, batch_size=512, epochs=500, validation_data=(X_test, y_test))
print(new_model.evaluate(X_validation, y_validation))
new_model.save_weights('new_weights.h5')

34/34 [==============================] - 1s 23ms/step - loss: 0.0264 - accuracy: 0.9925 - val_loss: 0.9405 - val_accuracy: 0.8987
Epoch 143/500
34/34 [==============================] - 1s 24ms/step - loss: 0.0282 - accuracy: 0.9904 - val_loss: 0.9019 - val_accuracy: 0.8933
Epoch 144/500
34/34 [==============================] - 1s 23ms/step - loss: 0.0293 - accuracy: 0.9914 - val_loss: 0.9240 - val_accuracy: 0.8933
Epoch 145/500
34/34 [==============================] - 1s 23ms/step - loss: 0.0267 - accuracy: 0.9910 - val_loss: 0.9098 - val_accuracy: 0.9024
Epoch 146/500
34/34 [==============================] - 1s 23ms/step - loss: 0.0285 - accuracy: 0.9918 - val_loss: 0.9429 - val_accuracy: 0.9005
Epoch 147/500
34/34 [==============================] - 1s 23ms/step - loss: 0.0324 - accuracy: 0.9892 - val_loss: 0.9248 - val_accuracy: 0.8969
Epoch 148/500
34/34 [==============================] - 1s 24ms/step - loss: 0.0292 - accuracy: 0.9906 - val_loss: 0.9566 - val_accuracy: 0.8879
Epoch 

In [21]:
files.download('new_weights.h5') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>